In [2]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from tqdm import tqdm

input_folder = r"G:\Public\CYGNSS"  # CYGNSS data path
output_folder = r"G:\Public\CYGNSS_clip_csv"  # Storage path for .csv file after cropping latitude and longitude

if not os.path.exists(output_folder):
    os.makedirs(output_folder)


# Calculate the coordinates of the corners of a square
def calculate_square(lat, lon, km_radius):
    earth_radius = 6371
    delta_lat = (km_radius / earth_radius) * (180 / np.pi)
    delta_lon = (km_radius / (earth_radius * np.cos(np.radians(lat)))) * (180 / np.pi)

    return {
        "top_left_lat": lat + delta_lat,
        "top_left_lon": lon - delta_lon,
        "top_right_lat": lat + delta_lat,
        "top_right_lon": lon + delta_lon,
        "bottom_left_lat": lat - delta_lat,
        "bottom_left_lon": lon - delta_lon,
        "bottom_right_lat": lat - delta_lat,
        "bottom_right_lon": lon + delta_lon
    }


for file_name in tqdm(os.listdir(input_folder)):
    if file_name.endswith(".nc") and "2023" in file_name:  # Process files from the year 2023
        file_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name.replace(".nc", "_clipped.csv"))

        dataset = xr.open_dataset(file_path)

        try:
            samples = dataset['sample'].values
            ddms = dataset['ddm'].values
            sp_lon = dataset['sp_lon'].values
            sp_lat = dataset['sp_lat'].values
            ddm_snr = dataset['ddm_snr'].values
            ptr = dataset['gps_tx_power_db_w'].values
            TxSP = dataset['tx_to_sp_range'].values
            SPRx = dataset['rx_to_sp_range'].values
            rec_antenna_gain = dataset['sp_inc_angle'].values
            GPS_antenna_gain = dataset['gps_ant_gain_db_i'].values
        except KeyError as e:
            print(f"Error: Missing variable in dataset - {e}")
            dataset.close()
            continue

        # Adjust longitude range: [0, 360] -> [-180, 180]
        sp_lon = np.where(sp_lon > 180, sp_lon - 360, sp_lon)

        # Filter sp_lon and sp_lat range: North Africa
        lon_min, lon_max = 10, 40
        lat_min, lat_max = 5, 20

        # Create a mask for filtering
        filtered_indices = (sp_lon >= lon_min) & (sp_lon <= lon_max) & \
                           (sp_lat >= lat_min) & (sp_lat <= lat_max)

        if not np.any(filtered_indices):
            print(f"No data in the specified region for {file_name}.")
            dataset.close()
            continue

        # Flatten multi-dimensional arrays for consistent filtering
        filtered_indices_flat = filtered_indices.flatten()
        filtered_sp_lon = sp_lon.flatten()[filtered_indices_flat]
        filtered_sp_lat = sp_lat.flatten()[filtered_indices_flat]
        filtered_samples = np.repeat(samples, sp_lon.shape[1])[filtered_indices_flat]
        filtered_ddms = np.tile(ddms, sp_lon.shape[0])[filtered_indices_flat]

        filtered_ddm_snr = ddm_snr.flatten()[filtered_indices_flat]
        filtered_ptr = ptr.flatten()[filtered_indices_flat]
        filtered_TxSP = TxSP.flatten()[filtered_indices_flat]
        filtered_SPRx = SPRx.flatten()[filtered_indices_flat]
        filtered_rec_antenna_gain = rec_antenna_gain.flatten()[filtered_indices_flat]
        filtered_GPS_antenna_gain = GPS_antenna_gain.flatten()[filtered_indices_flat]

        # Prepare data for DataFrame
        data = []
        for i in range(len(filtered_samples)):
            data.append([filtered_samples[i], filtered_ddms[i], filtered_sp_lon[i], filtered_sp_lat[i], 
                         filtered_ddm_snr[i], filtered_ptr[i], filtered_TxSP[i], filtered_SPRx[i], 
                         filtered_rec_antenna_gain[i], filtered_GPS_antenna_gain[i]])

        df = pd.DataFrame(data, columns=['sample', 'ddm', 'sp_lon', 'sp_lat', 'ddm_snr', 'gps_tx_power_db_w',
                                         'tx_to_sp_range', 'rx_to_sp_range', 'sp_inc_angle', 'gps_ant_gain_db_i'])

        # Calculate square quadrangle coordinates for filtered data
        km_radius = 1.5  # One side of the square is 3 kilometres long
        corners = df.apply(lambda row: calculate_square(row['sp_lat'], row['sp_lon'], km_radius), axis=1)

        if corners.empty:
            print("Warning: No corner coordinates generated.")
        else:
            corners_df = pd.DataFrame(corners.tolist())
            final_df = pd.concat([df, corners_df], axis=1)
            final_df.to_csv(output_path, index=False)
            #print(f"Data saved to {output_path}")

 16%|████████                                            | 1221/7869 [16:00<2:21:13,  1.27s/it]

No data in the specified region for cyg03.ddmi.s20231009-153200-e20231009-235959.l1.power-brcs.a31.d32.nc.


 21%|██████████▉                                         | 1654/7869 [27:40<2:19:01,  1.34s/it]

No data in the specified region for cyg08.ddmi.s20230808-000000-e20230808-235959.l1.power-brcs.a31.d32.nc.


100%|████████████████████████████████████████████████████| 7869/7869 [1:08:55<00:00,  1.90it/s]
